In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
# !sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
# !sudo apt-get install python-dev; pip install konlpy    
# !sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch
!pip install konlpy

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

torch.manual_seed(123)
np.random.seed(123)

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 128
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 100
learning_rate = 3e-5

In [ ]:
from konlpy.tag import Mecab
m = Mecab()
m.pos("농산물안전성연구")

In [6]:
# 학습용 데이터셋 불러오기
import pandas as pd
from konlpy.tag import Mecab
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 데이터 전처리
# train=train[['과제명', '요약문_연구내용','label']]
# test=test[['과제명', '요약문_연구내용']]
train.fillna('NAN', inplace=True)
test.fillna('NAN', inplace=True)

# train['data']=train['과제명']+train['요약문_연구내용']
# test['data']=test['과제명']+test['요약문_연구내용']

train['data']=train['과제명']+train['요약문_연구목표']+train['요약문_연구내용']+train['요약문_기대효과']+train['요약문_한글키워드']
test['data']=train['과제명']+train['요약문_연구목표']+train['요약문_연구내용']+train['요약문_기대효과']+train['요약문_한글키워드']

def clean_text(sents):
    sents_clean = []
    for sent in sents:
        sents_clean.append(re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]", " ", sent))
    return sents_clean


def tokenize(docs):
  mecab = Mecab()
  res=[]
  for doc in docs:
    tokenlist = mecab.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['NNG', 'NNP', 'NNB', 'NR']:
        temp.append(w[0])
    res.append(' '.join(temp))
  return res

# train['data'] = tokenize(clean_text(train['data']))
# test['data'] = tokenize(clean_text(test['data']))

train['data'] = clean_text(train['data'])
test['data'] = clean_text(test['data'])

# Train / Test set 분리
from sklearn.model_selection import train_test_split
trains, tests = train_test_split(train, test_size=0.2, random_state=123)
print("train shape is:", len(trains))
print("test shape is:", len(tests))


# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i]) for i in dataset['data']]
        self.labels = [np.int32(i) for i in dataset['label']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
        
data_train = BERTDataset(trains, 3, 2, tok, max_len, True, False)
data_test = BERTDataset(tests, 3, 2, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 46, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [15]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
from sklearn.metrics import f1_score, accuracy_score

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    #train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    train_f1 = f1_score(Y.cpu(), max_indices.cpu(), average='macro')
    #train_acc = accuracy_score(Y.cpu(), max_indices.cpu())
    return train_f1


In [10]:
del train, test, data_train, data_test, trains, tests

In [11]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [18]:
highest_acc = 0
patience = 0

# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    train_f1 = 0.0
    test_acc = 0.0
    test_f1 = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label) * 100
        # train_acc = calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            # print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # print("epoch {} train acc {}".format(e+1, train_acc))

    #테스트 문장 예측
    
    model.eval() # 평가 모드로 변경
    
    for test_batch_id, (test_token_ids, test_valid_length, test_segment_ids, test_label) in enumerate(tqdm_notebook(test_dataloader)):
        test_token_ids = test_token_ids.long().to(device)
        test_segment_ids = test_segment_ids.long().to(device)
        test_valid_length= test_valid_length
        test_label = test_label.long().to(device)
        test_out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label) * 100
        # test_acc = calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (test_batch_id+1)))
    # print("epoch {} test acc {}".format(e+1, test_acc))


    if test_acc - highest_acc >= 0.01:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_loss,
            }, './torchckpt/model.pt')
        patience = 0
    else:
        print("test acc did not improved. best:{} current:{}".format(highest_acc, test_acc))
        patience += 1
        if patience > 5:
            break
    print('current patience: {}'.format(patience))
    print("************************************************************************************")


/home/ubuntu/anaconda3/envs/lab17/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/1090 [00:00<?, ?it/s]

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
epoch 1 batch id 1 loss 3.7169153690338135 train acc 0.34266133637921187
epoch 1 batch id 101 loss 3.147700309753418 train acc 1.5780108060199012
epoch 1 batch id 201 loss 2.2499794960021973 train acc 3.243662046866429
epoch 1 batch id 301 loss 1.4606201648712158 train acc 4.117069056799198
epoch 1 batch id 401 loss 1.2927134037017822 train acc 4.591071891291251
epoch 1 batch id 501 loss 1.3525060415267944 train acc 4.868835766460174
epoch 1 batch id 601 loss 0.860675036907196 train acc 5.066803217070122
epoch 1 batch id 701 loss 0.9793289303779602 train acc 5.2018469414854485
epoch 1 batch id 801 loss 0.9309266209602356 train acc 5.418766935474618
epoch 1 bat

/home/ubuntu/anaconda3/envs/lab17/lib/python3.7/site-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/273 [00:00<?, ?it/s]

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
epoch 1 test acc 20.660522273425467


FileNotFoundError: [Errno 2] No such file or directory: 'torchckpt/model.pt'

In [ ]:
class BERTDataset1(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(0) for i in dataset['data']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
        #return (self.sentences[i] + (0, ))

    def __len__(self):
        return (len(self.sentences))


test_set = BERTDataset1(test, 3, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

checkpoint = torch.load('torchckpt/model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:


model.eval()


result = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      final = np.argmax(logits)
  result.append(final)

In [ ]:
print(result)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/sample_submission.csv')

submission['label'] = np.array(result)

submission.to_csv('/content/drive/MyDrive/Dacon/kobert_baseline.csv', index=False)